Python3引入的`装饰器`（Decorator）类似Java中的`注解`（Annotation），比如引用的时候都使用@符号前缀，可以拦截函数的执行过程，插入其他函数。也就是说可以更方便的实现AOP编程。**但是你不应该将其称之为Python的注解**，因为Python的装饰器在定义的时候，其理念还是和Java稍有不同的。

顾名思义，装饰器来自于GOF23种设计模式之一的`装饰器模式`。就是层层封装的技巧。可以回想一下Java中的IO库，就是采用的装饰器模式。另外如果你使用过Thrift，它的API也是装饰器模式的典型代表。

# 装饰器的定义与使用

In [1]:
def outer(func):
    def wrapper(*args, **kwargs):
        print("params:", *args)
        return func(*args, **kwargs)
    return wrapper
    
@outer
def add(a, b):
    return a + b


add(1, 2)

params: 1 2


3

## 带参数的装饰器

上面的装饰器都是不带参数的，如果要实现带参数的装饰器，就是要**多加一层def！**

In [12]:
def outer(name=''):
    def _outer(func):
        def wrapper(*args, **kwargs):
            print(name, "params:", *args)
            return func(*args, **kwargs)
        return wrapper
    return _outer

    
@outer("add")
def add(a, b):
    return a + b


add(1, 2)

add params: 1 2


3

# 装饰器实现AOP
来写一个简单的AOP。

In [3]:
def before_after(func):
    def wrapper(*args, **kwargs):
        print("before params:", *args)
        ret = func(*args, **kwargs)
        print("after return:", ret)
        return ret
    return wrapper

@before_after
def add(a, b):
    return a + b 

add(1, 2)

before params: 1 2
after return: 3


3

这个还比较初级，只是在调用前和调用后用print输出了一些信息，我们当然也可以wrapper内的func前后调用函数。 

来实现一个更通用的AOP

In [16]:
def aop(func_before=None, func_after=None):
    def _aop(func):
        def wrapper(*args, **kwargs):
            if func_before is not None:
                func_before(*args, **kwargs)
            ret = func(*args, **kwargs)
            if func_after is not None:
                func_after(ret)
            return ret
        return wrapper
    return _aop

def before(a:int, b:int):
    if a %2 == 1:
        a_p = "odd"
    else:
        a_p = "even"
    if b %2 == 1:
        b_p = "odd"
    else:
        b_p = "even"
    print("input : %d is %s, %d is %s " % (a, a_p, b, b_p))
    
def after(a:int):
    if a %2 == 1:
        a_p = "odd"
    else:
        a_p = "even"
    print("output: %d is %s" % (a, a_p))
    
@aop(before, after)
def add(a, b):
    return a + b 

add(1, 2)

input : 1 is odd, 2 is even 
output: 3 is odd


3

为了避免混淆，我把两个outer中的warpper分别命名为1和2，当然实际上是可以同名的。

In [4]:

def outer1(func):
    print("outer1")
    def wrapper1(*args, **kwargs):
        print("outer1 inner: " + func.__name__)
        return func(*args, **kwargs)
    return wrapper1

def outer2(func):
    print("outer2")
    def wrapper2(*args, **kwargs):
        print("outer2 inner: " + func.__name__)
        return func(*args, **kwargs)
    return wrapper2

@outer1
@outer2
def echo(a):
    print(a)
    
echo("test")

outer2
outer1
outer1 inner: wrapper2
outer2 inner: echo
test


In [5]:
from functools import wraps

def outer1(func):
    print("outer1")
    @wraps(func) 
    def wrapper1(*args, **kwargs):
        print("outer1 inner: " + func.__name__)
        return func(*args, **kwargs)
    return wrapper1

def outer2(func):
    print("outer2")
    @wraps(func) 
    def wrapper2(*args, **kwargs):
        print("outer2 inner: " + func.__name__)
        return func(*args, **kwargs)
    return wrapper2

@outer1
@outer2
def echo(a):
    print(a)
    
echo("test")

outer2
outer1
outer1 inner: echo
outer2 inner: echo
test


In [6]:
foo = outer1(outer2(echo))
print("xxx")
foo("test")

outer2
outer1
xxx
outer1 inner: echo
outer2 inner: echo
outer1 inner: echo
outer2 inner: echo
test


# 内置装饰器

​property
setter deleter 
类方法，静态方法，对象方法 TODO

@property

----------------
# 参考阅读
- [详解Python的装饰器](https://www.cnblogs.com/cicaday/p/python-decorator.html)